# Abschlussprojekt - Data Analysis and Digital Reporting with Python

### Imports
In der folgenden Zelle werden alle nötigen Librarires und Frameworks importiert. Neben den bekannten Packages der Vorlesung wurde des Weiteren noch das Package **dash_bootstrap_components** für die Dash-Entwicklung importiert. Das Package stellt Style-Patterns für html-Elemente zur Verfügung, wovon ich in meiner Dash-App gebraucht gemacht habe.

**Caution**  
Make sure to use dash_bootstrap-components is installed

`import sys`  
`!{sys.executable} -m pip install dash-bootstrap-components` 

In [1]:
#DATA OPERATIONS
import pandas as pd 
import numpy as np
import random

#DATA PLOTTING
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

#API
import yfinance as yf
import requests as re

#DASH
import dash
from jupyter_dash import JupyterDash 
from dash import dcc, html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc

# Aufgabe1

1. Aus Wikipedia werden die 101 Unternehmen aus dem S&P100 extrahiert und anschließend alphabetisch sortiert
2. Anhand der eindeutigen Zufallsziehung durch meine Matrikelnummer werden 20 zufällige aus dem S&P100 gefiltert und in dem Datensatz **df_sp20** gespeichert.

Ergebnisse:
- **df_sp20** = 20 Stocks aus S&P100 mit Spalten *Symbol, Name, Sector*

In [2]:
# 1. Download des S&P100
sp100url = 'https://en.wikipedia.org/wiki/S%26P_100'
df_sp100 = pd.read_html(sp100url)[2]
df_sp100 = df_sp100.reset_index()
df_sp100.sort_values(by=['Symbol'])

# 2. Reduzieren des S&P 100 auf 20 zufällige Unternehmen 
np.random.seed(103448) #Aufgrund von Duplikaten musste ich meine Matrikelnummer von 103446 auf 103448 erhöhen
twenty_randoms = np.random.randint(101, size=20)
df_sp20 = df_sp100.iloc[twenty_randoms]
df_sp20.sort_values(by=['Symbol'],ignore_index=True)

,index,Symbol,Name,Sector
0,3,ACN,Accenture,Information Technology
1,4,ADBE,Adobe,Information Technology
2,11,AXP,American Express,Financials
3,15,BKNG,Booking Holdings,Consumer Discretionary
4,17,BMY,Bristol Myers Squibb,Health Care
5,19,C,Citigroup,Financials
6,22,CL,Colgate-Palmolive,Consumer Staples
7,23,CMCSA,Comcast,Communication Services
8,26,COST,Costco,Consumer Staples
9,27,CRM,Salesforce,Information Technology


# Aufgabe 2
1. Zunächst werden die historischen Stockdaten (Open, High, Low, Adj Close, Volume) vom S&P100-Index (OEX) im Zeitraum 2015-2022 durch das yfinance package heruntergeladen und in dem Dataframe **df_sp100_stockdata** gespeichert Anschließend werden die Spalten *Open, High, Low, Adj Close und Volume* gefiltert

2. Für **df_sp100_stockdata** werden der discrete und daily return berechnet und in den Spalten *discrete_daily_return* und *log_daily_return* abgespeichert

3. Hier passieren die Punkte 1. und 2. gemeinsam für die einzelnen Stocks meines Datensatzes **df_sp20**. Für jedes der 20 Unternehmen werden die historischen Stockdaten heruntergeladen und anschließend die returns berechnet. Das Dictionary **dict_sp20_stockdata** speichert für jeden Ticker den jeweiligen Datensatz ab. BSP: {'NKE': df_NKE, 'AAPL': df_APPL}
 
Ergebnisse:

- **df_sp100_stockdata** = Stockdata vom S&P100 mit den Spalten *Open, High, Low, Adj Close, Volume, discrete_daily_return, log_daily_return*
- **dict_sp20_stockdata** = Dictionary für Stockdata von den 20 Unternehmen mit den Spalten *Open, High, Low, Adj Close, Volume, discrete_daily_return, log_daily_return*

In [3]:
# 1. Download von Stockdata des S&P100
df_sp100_stockdata = yf.download('^OEX', start="2015-01-01", end="2023-01-01")
df_sp100_stockdata = df_sp100_stockdata[['Open', 'High', 'Low', 'Adj Close', 'Volume']]

# 2) Berechnung von daily discrete und log return für den S&P100
df_sp100_stockdata['discrete_daily_return'] = df_sp100_stockdata['Adj Close'].pct_change()
df_sp100_stockdata['log_daily_return'] = np.log(df_sp100_stockdata['Adj Close']).diff()

# 3. Download Stockdata und Berechnung der returns für jedes Unternehmen in df_sp20
dict_sp20_stockdata = {}
for el in df_sp20['Symbol']:
    df_el = yf.download(el, start="2015-01-01", end="2023-01-01")
    df_el = df_el[['Open', 'High', 'Low', 'Adj Close', 'Volume']] 
    df_el['discrete_daily_return'] = df_el['Adj Close'].pct_change() 
    df_el['log_daily_return'] = np.log(df_el['Adj Close']).diff()
    dict_sp20_stockdata.update({el: df_el})

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

# Aufgabe 3

### Correlation & Volatility
1. Die Korrelation von den jeweiligen 20 Unternehmen mit dem S&P100 wird anhand des Adjusted Close Prices durchgeführt. Dabei ergab sich die Schwierigkeit, dass Unternehmen wie PayPal ihren IPO erst nach dem 01-01-2015 gehabt haben, sodass für die Zwischenzeit keine Korrelation durchgeführt werden kann

2. Die Volatilität wird berechnet durch die Standardabweichung des diskreten daily returns

Ergebnisse:
- **corr_on_oex** = Series für die Korrelationsmatrix von allen Daten meines Datensets sowie des S&P100 (OEX)
- **volas** = Series für die Volatilität Koeffizienten der einzelnen Unternehmen meines Datensets

In [4]:
# 1. Correlation
df_close = pd.DataFrame()
for el in dict_sp20_stockdata:
    df_close[el] = dict_sp20_stockdata[el]['Adj Close']
df_close['OEX'] = df_sp100_stockdata['Adj Close']
df_corr = df_close.corr(method='pearson')
corr_on_oex = df_corr['OEX'].drop('OEX')


# 2. Volatility --> UNTERSCHIED REALIZED VOLATILITY
dict_sp20_vola = {}
for el in dict_sp20_stockdata:
    vola = dict_sp20_stockdata[el]['discrete_daily_return'].std()
    dict_sp20_vola.update({el: vola})  
volas = pd.DataFrame.from_dict(dict_sp20_vola, orient='index').iloc[:, 0]

### Creating Portfolios
1. Die Funktion *createPortfolio(dict_indicator)* nimmt je nach Indicator ein Dictionary als Parameter und erstellt für die 10 niedrigsten und höchsten Koeffizienten jeweils ein Portfolio. In der Funktion werden mittels abs() alle negative Zahlen zu Positiven, um anschließend für ein Korrelationsportfolio die zehn größten Koeffizienten zu filtern, die die stärkste Korrelation aufweisen
2. Die oben definierte Funktion wird für jeden Portfolio Typ einmal aufgerufen

Ergebnisse:
- **df_p1_corr_low** = Portfolio der 10 Unternehmen mit geringster Korrelation zu S&P100
- **df_p2_corr_high** = Portfolio der 10 Unternehmen mit höchster Korrelation zu S&P100
- **df_p3_vola_low** = Portfolio der 10 Unternehmen mit niedrigster Volatilität
- **df_p4_vola_high** = Portfolio der 10 Unternehmen mit höchster Volatilität

In [5]:
# 1. Funktion zum Erstellen eines Portfolios
def createPortfolio(series_indicator):
    indicator_temp = abs(series_indicator) #only relevant for corr coefs
    df_p1_low_ten = indicator_temp.nsmallest(10)
    df_p2_high_ten = indicator_temp.nlargest(10)
    return df_p1_low_ten, df_p2_high_ten

# 2. Portfolios erstellen
df_p1_corr_low = createPortfolio(corr_on_oex)[0]
df_p2_corr_high = createPortfolio(corr_on_oex)[1]
df_p3_vola_low = createPortfolio(volas)[0]
df_p4_vola_high = createPortfolio(volas)[1]

### Calculate daily portfolio returns
Die Daily Portfolio Returns wurden berechnet anhand folgender Formel:

$$ R_p= \sum_i{w_i \cdot r_i} $$

- Wi: Definiert das dem Asset i zugeordnete Gewicht
- Ri: Return von Assets i

1. Die Funktion *calDailyPortflioReturns(df_portfolio)* errechnet die daily discrete und log returns. Unter 1.1 wird zuerst das {w_i} berechnet. Danach multiplziere ich {w_i} mit jedem daily return jedes Assets des Portfolios. Als letztes werden alle gewichteten Returns aller Assets summiert und als returns in **df_portfolio_daily_returns** gespeichert und zurückgegeben

2. Die oben definierte Funktion wird für jedes Portfolio aufgerufen und einer Variable zugewiesen


Ergebnisse Daily Returns:
- **df_p1_daily_returns** = Dataframe für P1 mit den Daily Portfolio Returns mit den Spalten *discrete_daily_return,log_daily_return*
- **df_p2_daily_returns** = für P2
- **df_p3_daily_returns** = für P3
- **df_p4_daily_returns** = für P4

In [6]:
# 1. Funktion zum Berechnen der Daily Portfolio Returns
def calDailyPortflioReturns(df_portfolio):
    # 1.1 Equal weight berechnen
    equal_weight = 1 / len(df_portfolio)
    df_portfolio_daily_returns = pd.DataFrame().reindex_like(df_sp100_stockdata[['discrete_daily_return', 'log_daily_return']])
    for asset in df_portfolio.index:
        df_stockdata = dict_sp20_stockdata[asset][['discrete_daily_return', 'log_daily_return']].copy()
        # 1.2 Für jedes Asset im Portfolio Daily Returns (discrete/log) gewichten 
        df_stockdata.loc[:, ['discrete_daily_return', 'log_daily_return']] *= equal_weight
        # 1.3 Die gewichteten Returns aller Assets summieren
        df_portfolio_daily_returns = df_portfolio_daily_returns.add(df_stockdata, fill_value=0)
    return df_portfolio_daily_returns

# 2. Die Daily Returns pro Portfolio berechnen
df_p1_daily_returns = calDailyPortflioReturns(df_p1_corr_low)
df_p2_daily_returns = calDailyPortflioReturns(df_p2_corr_high)
df_p3_daily_returns = calDailyPortflioReturns(df_p3_vola_low)
df_p4_daily_returns = calDailyPortflioReturns(df_p4_vola_high)

### Calculate monthly discrete portfolio returns
Um die monthly discrete portfolio returns zu bestimmen, wurden zunächst die täglichen Portfolio Values bestimmt. Dafür wurde folgende Formel aus dem Vorlesungsscript verwendet:

$$ PF_r= \sum_i{w_i \cdot S_i} $$

Die Gewichte sind in unserem Fall gleichgewichtet und für den Wert der Portfolio Assets S wurde der tägliche Adj Close Price verwendet. Für die monthly discrete portfolio returns wurde der letze Portfolio Value durch den ersten des Monats geteilt und abschließend noch 1 Minus gerechnet.

$$ r_t = \frac{S_t - S_{t-1}}{S_{t-1}} =\frac{S_t}{S_{t-1}} - 1 $$ 

1. Die Funktion *calPortfolioValue(df_portfolio)* errechnet für ein Portfolio die täglichen Portfolio Values aus. Dafür werden die gewichteten täglichen Adj Close summiert
2. Die Funktion *calMonthlyDiscretePortflioReturns(df_portfolio_daily_values)* errechnet mittels der täglichen Portfolio Values die monatlichen discrete Returns. Dafür wird der erste Portfolio Values des Monats durch den Letzten geteilt und anschließend noch 1 abgezogen
3. Die monthly discrete returns werden berechnet, indem die obigen Funktionen aufgerufen werden


Ergebnisse Monthly Discrete Returns:
- **df_p1_monthly_discrete_returns** = Monatliche discrete returns mit der Spalte discrete_monthly_return für P1
- **df_p2_monthly_discrete_returns** = "-" für P2
- **df_p3_monthly_discrete_returns** = "-" für P3
- **df_p4_monthly_discrete_returns** = "-" für P4
- **df_oex_monthly_discrete_returns** = "-" für S&P100 (Benötigt für den Switch daily/monthly in Aufgabe 4)

In [7]:
# 1. Funktion zum Bestimmen des Portfolio Values
def calPortfolioValue(df_portfolio):
    # 1.1 Equal weight berechnen
    equal_weight = 1 / len(df_portfolio)
    df_portfolio_daily_values = pd.DataFrame().reindex_like(df_sp100_stockdata[['Adj Close']])
    for asset in df_portfolio.index:
        df_stockdata = dict_sp20_stockdata[asset][['Adj Close']].copy()
        # 1.2 Für jedes Asset im Portfolio den Preis gewichten 
        df_stockdata.loc[:, ['Adj Close']] *= equal_weight
        # 1.3 Die gewichteten Preise aller Assets summieren
        df_portfolio_daily_values = df_portfolio_daily_values.add(df_stockdata, fill_value=0)
    return df_portfolio_daily_values


# 2. Funktion zum Berechnen der Monthly Discrete Portfolio Returns
def calMonthlyDiscretePortflioReturns(df_portfolio_daily_values):
    df_portfolio_daily_values['month'] = df_portfolio_daily_values.index.astype(str).str[:7] #filter months
    month_first_price = df_portfolio_daily_values.groupby('month').first() #get first close price of month
    month_last_price = df_portfolio_daily_values.groupby('month').last() #get last close price of month
    df_monthly_discrete_return = (month_first_price/month_last_price) - 1 #calculate discrete return
    df_monthly_discrete_return = df_monthly_discrete_return.rename(columns={'Adj Close': 'discrete_monthly_return'})
    return df_monthly_discrete_return

# 3. Berechnung der monthly discrete Returns für die Portfolios
df_p1_monthly_discrete_returns = calMonthlyDiscretePortflioReturns(calPortfolioValue(df_p1_corr_low))
df_p2_monthly_discrete_returns = calMonthlyDiscretePortflioReturns(calPortfolioValue(df_p2_corr_high))
df_p3_monthly_discrete_returns = calMonthlyDiscretePortflioReturns(calPortfolioValue(df_p3_vola_low))
df_p4_monthly_discrete_returns = calMonthlyDiscretePortflioReturns(calPortfolioValue(df_p4_vola_high))

# 4. Berechnung der monthly discrete returns für den S&P100
df_oex_monthly_discrete_returns = calMonthlyDiscretePortflioReturns(df_sp100_stockdata[['Adj Close']])


<ipython-input-7-b5152202b65e>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_portfolio_daily_values['month'] = df_portfolio_daily_values.index.astype(str).str[:7] #filter months


### Calculate monthly log portfolio returns
Um die monthly log portfolio returns zu bestimmen, wurden die daily log returns des jeweiligen Monats für das Portfolio aufsummiert 

$$ r_{0,n}^{log} = \sum_i{r_{i}^{log}} $$

1. Die Funktion *calMonthlyLogPortfolioReturns(df_portfolio_daily_returns)* filtert die daily portfolio log returns nach Monat und summiert diese anschließend auf und gibt 

Ergebnisse Monthly Log Returns:
- **df_p1_monthly_log_returns** = Monatliche log returns mit der Spalte discrete_monthly_return für P1
- **df_p2_monthly_log_returns** = "-" für P2
- **df_p3_monthly_log_returns** = "-" für P3
- **df_p4_monthly_log_returns** = "-" für P4

In [8]:
# 1. Funktion zum Berechnen der Monthly Log Portfolio Returns
def calMonthlyLogPortfolioReturns(df_portfolio_daily_returns):
    df_portfolio_monthly_log_returns = df_portfolio_daily_returns.copy()
    df_portfolio_monthly_log_returns['month'] = df_portfolio_monthly_log_returns.index.astype(str).str[:7]
    df_portfolio_monthly_log_returns = df_portfolio_monthly_log_returns[['log_daily_return','month']].groupby('month').sum()
    df_portfolio_monthly_log_returns = df_portfolio_monthly_log_returns.rename(columns={"log_daily_return": "log_monthly_return"})
    return df_portfolio_monthly_log_returns

# 2. Berechnung der Portfolio monthly log returns
df_p1_monthly_log_returns = calMonthlyLogPortfolioReturns(df_p1_daily_returns)
df_p2_monthly_log_returns = calMonthlyLogPortfolioReturns(df_p2_daily_returns)
df_p3_monthly_log_returns = calMonthlyLogPortfolioReturns(df_p3_daily_returns)
df_p4_monthly_log_returns = calMonthlyLogPortfolioReturns(df_p4_daily_returns)

# Aufgabe 4

### Download von Stock Informationen
Um die Stockinformationen (Logo, Sector, Mitarbeiteranzahl, Websitelink, Market Cap) zu erhalten, nutze ich die API FinancialModelingPrep. Die Informationen werden für jedes Unternehmen in **dict_cmpny_info** gespeichert

Ergebnisse:
- **dict_cmpny_info** = Dictionary, dass für jedes der 20 Unternehmen ein Dataframe mit den Spalten *companyName, image, sector, fullTimeEmployees, mktCap, website*

In [9]:
#API-Settings: General Settings
api_key = '094b2c1b0f6740addb83d7248f0f4c60'
key_parameter = {'apikey': api_key}
fmp_url = 'https://financialmodelingprep.com/api/v3'

#API GET COMPANY INFORMATION
dict_cmpny_info = {}
for el in df_sp20['Symbol']:
    res_info = re.get(fmp_url + '/' + 'profile' + '/' + el , params=key_parameter)
    df_cpmny_info = pd.DataFrame.from_dict(res_info.json())
    df_cpmny_info = df_cpmny_info[['companyName', 'image', 'sector','fullTimeEmployees', 'mktCap', 'website']]
    df_cpmny_info.reset_index(inplace=True)
    dict_cmpny_info.update({el: df_cpmny_info})

### Berechnung der realisierten Volatiliät und Sharpe Ratio pro Jahr 
1. Die Funktion *calDailySharpeRatioWithBenchmark()* berechnet die Daily Sharpe Ratio mit einer Risk Free Rate von 0.0 und der Benchmark des S&P100 Index. 
2. Die Funktion *calVolaAndSharpeRatioPerYear()* berechnet die jährliche Volatilität und Sharpe Ratio. Dafür werden die tägliche Volatilität und Sharpe Ratio mit 252**(1/2) multipliziert

Ergebnisse:
- **df_p1_yearly_sr_vola** = Dataframe für die jährliche Volatilität und Sharpe Ration von P1
- **df_p2_yearly_sr_vola** = "-" von P2
- **df_p3_yearly_sr_vola** = "-" von P3
- **df_p4_yearly_sr_vola** = "-" von P4

In [10]:
# 1. Funktion zu Berechnung der Sharpe Ratio mit Benchmark 
def calDailySharpeRatioWithBenchmark(portfolio, risk_free_rate=0.0):
    benchmark = df_sp100_stockdata
    
    # Calculate Average Daily Return
    mean_portfolio_daily_return = portfolio['discrete_daily_return'].mean()
    mean_benchmark_daily_return = benchmark['discrete_daily_return'].mean()
    
    # Calculate The Excess Returns 
    excess_return = mean_portfolio_daily_return - mean_benchmark_daily_return
    
    # Calculate the Standard Deviation
    std_portfolio = portfolio['discrete_daily_return'].std()

    # Calculate Daily Sharpe Ratio with Benchmark
    daily_sharpe_ratio_with_benchmark = (excess_return - risk_free_rate) / std_portfolio
    return daily_sharpe_ratio_with_benchmark


# 2. Funktion zur Berechnung der Volatiliät und Sharpe Ratio pro Jahr
def calVolaAndSharpeRatioPerYear(df_portfolio):
    portfolio = df_portfolio.copy()
    portfolio['Year'] = portfolio.index.astype(str).str[:4]
    years = portfolio['Year'].unique()
    df_vola_sr = pd.DataFrame(columns = ['Year', 'Volatility', 'Sharpe Ratio Benchmark S&P100'])
    for year in years:
        vola_yearly = 252**(1/2) * portfolio[portfolio['Year'] == year]['discrete_daily_return'].std()
        #sr_yearly = 252**(1/2) * calSharpeRatio(portfolio[portfolio['year'] == year])
        sr_yearly_benchmark = 252**(1/2) * calDailySharpeRatioWithBenchmark(portfolio[portfolio['Year'] == year])
        new_row = pd.DataFrame({'Year':year, 'Volatility':vola_yearly, 'Sharpe Ratio Benchmark S&P100':sr_yearly_benchmark}, index=[0])
        df_vola_sr = pd.concat([new_row,df_vola_sr.loc[:]]).reset_index(drop=True)
    df_vola_sr = df_vola_sr.sort_values(by=['Year'],ignore_index=True)
    return df_vola_sr

df_p1_yearly_sr_vola = calVolaAndSharpeRatioPerYear(df_p1_daily_returns)
df_p2_yearly_sr_vola = calVolaAndSharpeRatioPerYear(df_p2_daily_returns)
df_p3_yearly_sr_vola = calVolaAndSharpeRatioPerYear(df_p3_daily_returns)
df_p4_yearly_sr_vola = calVolaAndSharpeRatioPerYear(df_p4_daily_returns)

### Sharpe Ratio für alle Unternehmen
Die Funktion *calSharpeRatio()* berechnet die durschnittliche jährliche Sharpe Ratio eines Unternehmens. Diese Information nutze ich bei der Unternehmenansicht nach der Dropdown Auswahl

Ergebnisse:
- **cmpny_sr** = Durschnittliche jährliche Sharpe Ratio aller 20 Unternehmen meines Datensets

In [11]:
#Funktion für die durchschnittliche järhliche Sharpe Ratio
def calSharpeRatio(data, risk_free_rate=0.0):
    # Calculate Average Daily Return
    mean_daily_return = data.mean()
    # Calculate Standard Deviation
    std = data.std()
    # Calculate Daily Sharpe Ratio
    daily_sharpe_ratio = (mean_daily_return - risk_free_rate) / std
    # Annualize Daily Sharpe Ratio
    sharpe_ratio = 252**(1/2) * daily_sharpe_ratio
    return sharpe_ratio


# 2. Berechnung der Sharpe Ratio pro Unternehmen meines Datensets
dict_sp20_sr = {}
for el in dict_sp20_stockdata:
    sr = calSharpeRatio(dict_sp20_stockdata[el]['discrete_daily_return'])
    dict_sp20_sr.update({el: sr})  
cmpny_sr = pd.DataFrame.from_dict(dict_sp20_sr, orient='index').iloc[:, 0]

# Dash

### CSS Styling
In der folgenden Zelle werden Styling-Elemente definiert, die später für das Attribut *style* von Dash-Komponenten eingesetzt werden. 
- **margin_style** definiert Margins und zentriert innere Elemente. 
- **img_style** definiert die Größe eines Images, welches seinen Einsatz in der Darstellung der Firmenlogos hat

In [12]:
margin_style = {
    "margin-top": "50px",
    "margin-bottom": "50px",
    "textAlign": "center"
}

img_style = {
    "display": "block",
    "margin-left": "auto",
    "margin-right": "auto",
    "width":"50%",
    "height":"50%",
}

### Dash Komponten - Summary Tab
In den folgenden Zeilen werden Funktionen definiert, die Dash Komponenten für das Summary kreieren und zurückgeben. Die Definition der Funktionen half mir die Übersichtlichkeit des Dash Codes zu bewahren und den Code nach Funktionalität zu ordnen und zu schachteln.

Ergebnisse:
- **createCheckbox()** = gibt Dash Checkboxes für den S&P100 Trace und die Montly Returns zurück
- **createLinePlot(sp=0, monthly=0)** = Gibt je nach Checkbox Eingabe ein Lineplot für daily/monthly returns der 4 Portfolios zurück
- **createHisto(portfolio)** = gibt ein Histogramm für die Verteilung der daily returns der Portfolios zurück
- **createIndicatorTable(df)** = gibt eine Tabelle für die jährlich berechnete Volatilität und Sharpe Ratio eines Portfolios zurück

In [13]:
def createCheckbox():
    checklist = html.Div(
    [
        dbc.Label("Advanced Settings"),
        dbc.Checklist(options=[
            {"label": "Add S&P100", "value": 1},
        ],id="checklist-input"),
        dbc.Checklist(options=[
            {"label": "Montly instead of daily returns", "value": 1},
        ],id="checklist2-input"),
    ])
    return checklist

In [14]:
#Line Plot
def createLinePlot(sp=0, monthly=0):
    fig_line = make_subplots(rows=1, cols=1, x_title='Time')
    if monthly == 1:
        fig_line.add_trace(go.Scatter(name="P1 Low Corr",x=df_p1_monthly_discrete_returns.index, y=df_p1_monthly_discrete_returns['discrete_monthly_return']))
        fig_line.add_trace(go.Scatter(name="P2 High Corr",x=df_p2_monthly_discrete_returns.index, y=df_p2_monthly_discrete_returns['discrete_monthly_return']))
        fig_line.add_trace(go.Scatter(name="P3 Low Vol",x=df_p3_monthly_discrete_returns.index, y=df_p3_monthly_discrete_returns['discrete_monthly_return']))
        fig_line.add_trace(go.Scatter(name="P4 High Vol",x=df_p4_monthly_discrete_returns.index, y=df_p4_monthly_discrete_returns['discrete_monthly_return']))
        fig_line.add_trace(go.Scatter(name="S&P 100",x=df_oex_monthly_discrete_returns.index, y=df_oex_monthly_discrete_returns['discrete_monthly_return']))
        fig_line.update_layout(title="Monthly Returns 2015-2022", yaxis_title="Discrete Monthly Return")
        
    else:
        fig_line.add_trace(go.Scatter(name="P1 Low Corr",x=df_p1_daily_returns.index, y=df_p1_daily_returns['discrete_daily_return']))
        fig_line.add_trace(go.Scatter(name="P2 High Corr",x=df_p2_daily_returns.index, y=df_p2_daily_returns['discrete_daily_return']))
        fig_line.add_trace(go.Scatter(name="P3 Low Vol",x=df_p3_daily_returns.index, y=df_p3_daily_returns['discrete_daily_return']))
        fig_line.add_trace(go.Scatter(name="P4 High Vol",x=df_p4_daily_returns.index, y=df_p4_daily_returns['discrete_daily_return']))
        fig_line.add_trace(go.Scatter(name="S&P 100",x=df_sp100_stockdata.index, y=df_sp100_stockdata['discrete_daily_return']))
        fig_line.update_layout(title="Daily Returns 2015-2022", yaxis_title="Discrete Daily Return")

    if sp == 1:
        fig_line.data[4].visible=True
    else:
        fig_line.data[4].visible=False
    
    fig_line.update_layout(
        plot_bgcolor='rgba(0, 0, 0, 0)',
        paper_bgcolor='rgba(0, 0, 0, 0)',
        font_color="white",
        title_x=0.5,
        title_font_color="white",
        legend_title_font_color="white")
    line_plot = dcc.Graph(figure=fig_line)
    return line_plot

In [15]:
def createHisto(portfolio):
    fig_hist = px.histogram(portfolio, x="discrete_daily_return", title='My title')
    fig_hist.update_layout(
        title_text='Distribution Of Returns', 
        title_x=0.5,
        plot_bgcolor='rgba(0, 0, 0, 0)',
        paper_bgcolor='rgba(0, 0, 0, 0)',
        font_color="white",
        title_font_color="white",
        legend_title_font_color="white")
    histo_plot = dcc.Graph(figure=fig_hist)
    return histo_plot

In [16]:
def createIndicatorTable(df):
    table = dbc.Table.from_dataframe(df, striped=True, bordered=True, hover=True)
    return table

### Dash Komponten - Portfolio Tab
In den folgenden Zeilen werden Funktionen definiert, die Dash Komponenten für die Portfolio Tabs kreieren und zurückgeben. 

Ergebnisse:
- **createListGroup(df_portfolio)** = gibt es Dash Liste für die Unternehmen eines Portfolios zurück
- **createDropdown(portfolio)** gibt ein Dash Dropdown Menu für alle Unternehmen meines Datensets zurück
- **checkPortfolioMembership(cmpny, portfolio_type)** = gibt je nach Company und Portfoliotyp (Corr/Vola) die Portfolio Memebership als String zurück (BSP: P1 - Corr Lower Half)
- **createCmpnyInfo(cmpny, portfolio_type)** = gibt je nach Company und Portfoliotyp eine Dash Card mit den  geforderten Informationen der Unternehmen (Sector, Number of employees, etc.) zurück

In [17]:
#LIST portfolio
def createListGroup(df_portfolio):
    list_items = []
    for el in df_portfolio.index:
        item = dbc.ListGroupItem(el)
        list_items.append(item)
    return dbc.ListGroup(children=list_items)

In [18]:
#DROPDOWN
def createDropdown(portfolio):
    select_options = []
    for stock in portfolio:
        option = {"label": stock, "value": stock}
        select_options.append(option)

    select = dbc.Select(id="select", options=select_options, placeholder="Please select a ticker")
    dropdown = html.Div(select, style={'margin-bottom': "20px"})
    return dropdown

In [19]:
def checkPortfolioMembership(cmpny, portfolio_type):
    if portfolio_type == "p_corr":
        if cmpny in df_p1_corr_low:
            return "P1 - Corr Lower Half "
        elif cmpny in df_p2_corr_high:
            return "P2 - Corr Upper Half "
    else:
        if cmpny in df_p3_vola_low:
            return "P3 - Vola Lower Half "
        elif cmpny in df_p4_vola_high:
            return "P4 - Vola Upper Half "

In [20]:
def createCmpnyInfo(cmpny, portfolio_type):
    if cmpny == None:
        return dbc.Card()
    card = dbc.Card(
    [
        dbc.Row(
            [
                dbc.Col(html.Img(src=dict_cmpny_info[cmpny]["image"].values[0], style=img_style),className="col-md-4"),
                dbc.Col(
                    dbc.CardBody(
                        [
                            html.H4(dict_cmpny_info[cmpny]["companyName"], className="card-title"),
                            html.P("Sector: " + dict_cmpny_info[cmpny]["sector"],className="card-text"),
                            html.P("Number of employees: " + dict_cmpny_info[cmpny]["fullTimeEmployees"],className="card-text"),
                            html.P('Market capitalisation: ' +dict_cmpny_info[cmpny]['mktCap'].apply(str) ,className="card-text"),
                            html.P("Sharpe Ratio (Yearly Mean): " + str(cmpny_sr[cmpny]) ,className="card-text"),
                            html.P("Portfolio Membership: " + checkPortfolioMembership(cmpny, portfolio_type) ,className="card-text"),
                            dbc.Button("Website", color="primary", href=dict_cmpny_info[cmpny]["website"].values[0]),
                        ]
                    ), className="col-md-8"
                ),
            ],
            className="g-0 d-flex align-items-center",
        )
    ], 
    className="mb-3", style={"maxWidth": "540px"})
    return card

### Main Dash App
Je nach ausgewähltem Tab wird die Funktion createTabContent(active_tab) aufgerufen. Diese rendert je nach Tab den entsprechenen Dash Code und nimmt Gebrauch der oben definierten Datensätze und Funktionen für die Dash Komponenten

In [21]:
def createTabContent(active_tab):
    if active_tab == "p_corr":
        lower_half = df_p1_corr_low
        upper_half = df_p2_corr_high
    elif active_tab == "p_vola":
        lower_half = df_p3_vola_low
        upper_half = df_p4_vola_high
    else:
        content = dbc.Container([
            html.H2("Returns of all 4 portfolios", style=margin_style),
            createCheckbox(),
            html.Div([
                createLinePlot()
            ], id='line-plot'),
            dbc.Row([
                html.H2("Portfolio 1 - Lowest Correlation", style=margin_style),
                dbc.Col(createHisto(df_p1_daily_returns), width=6),
                dbc.Col([
                    html.Br(),
                    html.Small("Volatility & Sharpe Ratio per year", className="card-text text-muted"),
                    html.Br(),
                    createIndicatorTable(df_p1_yearly_sr_vola)
                ], width=6)
            ]),
            dbc.Row([
                html.H2("Portfolio 2 - Highest Correlation", style=margin_style),
                dbc.Col(createHisto(df_p2_daily_returns), width=6),
                dbc.Col([
                    html.Br(),
                    html.Small("Volatility & Sharpe Ratio per year", className="card-text text-muted"),
                    html.Br(),
                    createIndicatorTable(df_p2_yearly_sr_vola)
                ], width=6)
            ]),
            dbc.Row([
                html.H2("Portfolio 3 - Lowest Volatility", style=margin_style),
                dbc.Col(createHisto(df_p3_daily_returns), width=6),
                dbc.Col([
                    html.Br(),
                    html.Small("Volatility & Sharpe Ratio per year", className="card-text text-muted"),
                    html.Br(),
                    createIndicatorTable(df_p3_yearly_sr_vola)
                ], width=6)
            ]),
            dbc.Row([
                html.H2("Portfolio 4 - Highest Volatility", style=margin_style),
                dbc.Col(createHisto(df_p4_daily_returns), width=6),
                dbc.Col([
                    html.Br(),
                    html.Small("Volatility & Sharpe Ratio per year", className="card-text text-muted"),
                    html.Br(),
                    createIndicatorTable(df_p4_yearly_sr_vola)
                ], width=6)
            ]),
            dbc.Row([
                html.H2("Interpretation", style=margin_style),
                html.P("In the line plot of the daily returns from 2015 to 2022 you can see that the 4 different portfolios perform similarly and there are no major outliers. Also if you add the returns of the S&P100 to the graph, you can see that this graph is also very similar to the portfolios. What you can see is that the portfolio 3 (Lowest Volatility) is more constant and the variance of the returns is lower than the other portfolios. This can also be seen well in the tables of the portfolios, which list the volatility of the individual years. There you can see that the annual volatility is mostly below that of the other portfolios. The Sharpe ratio to the benchmark compares the risk-adjusted performance of the portfolio to the benchmark. The higher the Sharpe Ratio, the better the performance. A Sharpe Ratio greater than 1 indicates that the portfolio has outperformed the benchmark. If it is less than 1, this represents underperformance. All portfolios here vary in Sharpe Ratio over the years to benchmark the S&P100 and show both temporary outperformance and underperformance.")
            ]),
        ])
        return content
    content = dbc.Container([
            dbc.Row([
                html.H2("List of lower and upper half", style=margin_style),
                dbc.Col([
                    html.Small("Lower Half", className="card-text text-muted"),
                    createListGroup(lower_half)],
                    width=6
                ),
                dbc.Col([
                        html.Small("Upper Half", className="card-text text-muted"),
                        createListGroup(upper_half)],
                        width=6
                )]
            ),
            html.H2("Dropdown Menu for Company Information", style=margin_style),
            dbc.Row([
                dbc.Col(width=3),
                dbc.Col(createDropdown(dict_sp20_stockdata), width=6),
                dbc.Col(width=3),
            ]),
            dbc.Row([
                dbc.Col(width=3),
                dbc.Col(html.Div(id="cmpny_info"),width=6),
                dbc.Col(width=3)
            ])
    ])
                
        
    return content

In [ ]:
app = dash.Dash(external_stylesheets=[dbc.themes.DARKLY])
app.layout = dbc.Container(
    [
        html.H1("Abschlussprojekt - Data Analysis and Digital Reporting with Python"),
        html.Hr(),
        dbc.Tabs(
            [
                dbc.Tab(label="Summary", tab_id="summary"),
                dbc.Tab(label="Correlation Portfolio", tab_id="p_corr"),
                dbc.Tab(label="Volatility Portfolio", tab_id="p_vola"),
            ],
            id="tabs",
            active_tab="summary",
        ),
        html.Div(id="tab-content", className="p-4"),
    ]
)

#USER INPUTS
@app.callback(
    Output("tab-content", "children"),
    Input("tabs", "active_tab"),
)
def showTabContent(active_tab):
    return createTabContent(active_tab)
        
    
@app.callback(
    Output("cmpny_info", "children"), [Input("select", "value"), Input("tabs", "active_tab")]
)
def showCmpnyInfo(cmpny, portfolio_type):
    return createCmpnyInfo(cmpny, portfolio_type)
        

@app.callback(
    Output("line-plot", "children"),
    [
        Input("checklist-input", "value"),
        Input("checklist2-input", "value")
    ],
)
def updateLinePlot(checklist_value, checklist2_value):
    if checklist_value != None:
        n_checks = len(checklist_value)
    else:
        n_checks = 0
    if checklist2_value != None:
        n_checklist2 = len(checklist2_value)
    else:
        n_checklist2 = 0
    return createLinePlot(sp=n_checks, monthly=n_checklist2)
    
app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
